In [1]:
import findspark
findspark.init("/home/triloki/spark-2.4.6-bin-hadoop2.7")

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data.show(3)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
+----+--------------------+
only showing top 3 rows



In [6]:
#renaming column
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [7]:
data.show(3)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows



## Clean and Prepare the Data

In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length',length(data['text']))

In [10]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [11]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [12]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### Model


In [15]:
from pyspark.ml.classification import NaiveBayes

In [16]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [19]:
cleaner = data_prep_pipe.fit(data)

In [20]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [21]:
clean_data = clean_data.select(['label','features'])

In [22]:
clean_data.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
+-----+--------------------+
only showing top 3 rows



In [23]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [24]:
spam_predictor = nb.fit(training)

In [24]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [25]:
test_results = spam_predictor.transform(testing)

In [26]:
test_results.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-798.53005289200...|[1.0,3.3064290841...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1067.3008251001...|[1.0,4.4685119913...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-569.99437532035...|[1.0,7.7691965590...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-834.42671861706...|[1.0,1.5316073022...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-806.70108961938...|[1.0,9.6455993559...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model: {}".format(acc))

Accuracy of model: 0.9266453662190148
